## Making the text processor
**STEPS**:
- First the class Data Structure is returned 
- Each class may have multiple instances in the boxes_dict 
- Let us say that we have a class in the dictionary with the Confidence value for each box and the (xo,yo,w,h) of the box 
- For each class there will be a **list**
- Each list element -
    - Contains the confidence of the Box 
    - Box drawing information

In [2]:
%cd files

D:\DATA SCIENCE\RAAHI\YOLO\files


In [64]:
#%%
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
import glob
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

from yolo_class import Yolo_v3 
from loaders import * 
from drawers import * 

if type(tf.contrib) != type(tf): 
    tf.contrib._warning = None
    
_BATCH_NORM_DECAY = 0.9
_BATCH_NORM_EPSILON = 1e-05
_LEAKY_RELU = 0.1
_ANCHORS = [(10, 13), (16, 30), (33, 23),
            (30, 61), (62, 45), (59, 119),
            (116, 90), (156, 198), (373, 326)]
_MODEL_SIZE = (416, 416)

PATH = '../Validation Set'
img_names = [PATH + '/'+str(i)+'.jpg' for i in range(1,4)]

tf.reset_default_graph()
batch_size = len(img_names)
batch = load_images(img_names, model_size=_MODEL_SIZE)
class_names = load_class_names('coco.names')
n_classes = len(class_names)
max_output_size = 10
iou_threshold = 0.5
confidence_threshold = 0.5

model = Yolo_v3(n_classes=n_classes, model_size=_MODEL_SIZE,
                max_output_size=max_output_size,
                iou_threshold=iou_threshold,
                confidence_threshold=confidence_threshold)

inputs = tf.placeholder(tf.float32, [batch_size, 416, 416, 3])

detections = model(inputs, training=False)

model_vars = tf.global_variables(scope='yolo_v3_model')
assign_ops = load_weights(model_vars, 'yolov3.weights')


#%%

 
#%%

In [66]:
with tf.Session() as sess:
    sess.run(assign_ops)
    detection_result = sess.run(detections, feed_dict={inputs: batch})
# draw_boxes(img_names, detection_result, class_names, _MODEL_SIZE)

In [67]:
file_ind = open("../../Data/REQUIRED_NAMES.txt",'r')
all_file = open("coco.names")
# read 
indoor_names = file_ind.read().splitlines()
all_names = all_file.read().splitlines()
ind_name, all_name = {},{}

for j,k in enumerate(all_names):
    all_name[j] = k
    if(k in indoor_names):
        ind_name[j] = k
# print(ind_name,all_name)

# make an empty list for each label 
store_info = []

In [8]:
# process the dictionary image by image

In [68]:
print("People in image 3 :",detection_result[2][0])

People in image 3 : [[ 75.40086   132.6846     96.079544  213.20496     0.9376799]
 [239.32097   137.81238   245.2517    156.26239     0.9088346]
 [306.77042   135.80153   353.52597   357.20856     0.866853 ]
 [252.65015   132.3807    262.26837   162.12482     0.8539324]]


In [78]:
store_info = []
def get_orientation(x1,x2,center):
    '''Returns orientation of the object with some 
       threshold'''
    
    object_mid = (x1+x2)//2 
    
    thres = center//10
    
    left = center - thres 
    right = center + thres 
    
    if(object_mid < left):
        return "left" 
    elif(object_mid > right):
        return "right" 
    else:
        return "ahead"

#process the class
def process_class(boxes):
    # boxes must contain elements like (xo,yo,w,h)
    c = 0
    result = []
    for box in boxes:
        xy,confidence = box[:4],box[4]
        # to do...
        
        '''xy[0],xy[1] : (x,y) : top left coordinate 
           xy[2],xy[3] : (x,y) : bottom right coordinate'''
        top_left = (xy[0],xy[1])
        bottom_right = (xy[2],xy[3])
        
        '''Find the width and height of the bounding box'''
        width = abs(top_left[0] - bottom_right[0])
        height = abs(top_left[1] - bottom_right[1])
        
        # Find if the object is towards left, right or the center 
        x1,x2 = top_left[0],bottom_right[0]
        orient = get_orientation(x1,x2,416//2)
        
        # feed these into the distance formula given 
        D = ((2*3.14*180)/(width + height*360))* 1000 + 3
        # result contains elements with -
        #  1. how good is that prediction
        #  2. how close is that prediction 
        # 3. where is that prediction
        result.append((confidence/D,round(D/12,2),orient))
    # sort on the cofidence/distance ratio 
    result = sorted(result,reverse = True)
    return result[:2]

# process dictionary 
def process_box_dict(boxes_dict):
    for c_index,boxes in boxes_dict.items():
        # if the class name is not in the required names 
        if c_index not in ind_name:
            continue
        if(len(boxes) == 0):
            continue 
        else:
            class_name = ind_name[c_index]
            # now return the 2 highest confidence object and the nearest
            result = process_class(boxes)
            # this distance var contains the distance to the class 
            # in inches 
            
            for k in result:
                distance = k[1]
                text = "There is a "+class_name+" "+str(distance)+" feet ahead of you"
                store_info.append((distance,text))
    if len(store_info) == 0:
        store_info.append("There are no obstacles near you!")
    

## Steps 
- Two highest **probability / distance** objects 
- Remove objects more than 12 feet.
- Link openCV image to the yolo 

In [79]:
store_info =[]
process_box_dict(detection_result[0])

In [80]:
store_info

[(3.77, 'There is a person 3.77 feet ahead of you'),
 (4.84, 'There is a person 4.84 feet ahead of you'),
 (4.34, 'There is a pottedplant 4.34 feet ahead of you')]

## To - Do 
- Link openCV image to this script and run examples